In [1]:
import glob 
import os
from PIL import Image
import matplotlib.pyplot as plt
from PIL import ImageFile
import numpy as np
import json
%matplotlib inline
from tqdm import tqdm

In [2]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
di_class = ['Ab01','Ab02','Ab03','Ab04','Ab05','Ab06','Ab07','Ab08','Ab09','Ab10']

In [196]:
di = 'Mu01' 
img_dir_path = 'C:/Users/seonh/Downloads/062.반려동물 질병 진단을 위한 영상 데이터 (근골격계)/01.데이터/Training/원천데이터/' + di
#img_dir_path = 'C:/Users/seonh/Downloads/062.반려동물 질병 진단을 위한 영상 데이터 (근골격계)/01.데이터/Validation/원천데이터/sep/' + di

In [197]:
json_dir_path = 'C:/Users/seonh/Downloads/062.반려동물 질병 진단을 위한 영상 데이터 (근골격계)/01.데이터/Training/라벨링데이터/TL2_DOG'
#json_dir_path = 'C:/Users/seonh/Downloads/062.반려동물 질병 진단을 위한 영상 데이터 (근골격계)/01.데이터/Validation/라벨링데이터/VL2_DOG'
json_name_list = os.listdir(json_dir_path)  # 다 영어로..?

In [5]:
print(len(json_name_list))

113725


In [6]:
print(json_name_list[:10])

['D_62_19980513_IF_0028_NOR_Mu01_20211216_0165.json', 'D_62_19980513_IF_0028_NOR_Mu01_20211216_0167.json', 'D_62_19980513_IF_0028_NOR_Mu01_20211216_0168.json', 'D_62_19980513_IF_0028_NOR_Mu01_20211216_0169.json', 'D_62_19980615_IF_0028_NOR_Mu07_20211216_0171.json', 'D_62_19980623_IM_0028_NOR_Mu01_20211216_0173.json', 'D_62_19980623_IM_0028_NOR_Mu01_20211216_0174.json', 'D_62_19980623_IM_0028_NOR_Mu01_20211216_0175.json', 'D_62_19980623_IM_0028_NOR_Mu01_20211216_0176.json', 'D_62_19980623_IM_0028_NOR_Mu01_20211216_0177.json']


In [141]:
def padding(img_path):
    img = Image.open(img_path)
    img = img.convert('RGB')
    w, h = img.size
    img = np.array(img)
    
    margin = [np.abs(w - h) // 2 , np.abs(w - h) // 2]
    if w > h:
        margin_list = [margin, [0,0]]
    else:
        margin_list = [[0,0], margin]
        
    if len(img.shape) == 3:
        margin_list.append([0,0])
        
    output = np.pad(img, margin_list, mode='constant')
    img = Image.fromarray(output)
    # 이미지 크기
    img = img.resize((224, 224))
    #img = np.array(img)
    return img

In [8]:
def status_to_int(status):
    if status == 'NOR': #정상
        int_status = 0
    else:
        int_status = 1
    return int_status

In [9]:
def json_name_to_img_path(json_name):
    img_name = json_name.split('.')[0] +'.jpg'
    img_path = img_dir_path + '/'+ img_name
    return img_path

In [149]:
from random import randrange

In [ ]:
err = 0
abb = 0
numb = 0
x_data, y_data = [], []
resize = 224
drop_t = []

rot = [-5, 5]
trans = [[resize/10, 0], [-(resize/10), 0], [0,(resize/10)], [0,-(resize/10)] ]
    
for json_name  in tqdm(json_name_list):
    if json_name.split('_')[6] == di:
        json_path = json_dir_path + '/'+ json_name
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                status = data["metadata"]['Disease']
                int_status = status_to_int(status)
                
                #if int_status == 1:
                    
                y_data.append(int_status)
                    
            img_path = json_name_to_img_path(json_name)
            img = padding(img_path)    
            img_f = np.array(img)
            x_data.append(img_f)
            
            numb +=1
            
            if y_data[-1] == 1:
                abb +=1
            else:
                drop_t.append(numb)
                
                
                for angle in rot:   #test_img = test_img.rotate(-5)
                    ss = randrange(len(x_data)+1)
                    y_data.insert(ss, y_data[-1])
                    
                    rot_img = img.rotate(angle)
                    np_img = np.array(rot_img)
                    x_data.insert(ss, np_img)
                    #train_set.append(value)
                    
                for k in trans:
                    ss = randrange(len(x_data)+1)
                    y_data.insert(ss, y_data[-1])
    
                    tran_img = img.rotate(0, translate = k)
                    np_img = np.array(tran_img)
                    x_data.insert(ss, np_img)            
                
        except FileNotFoundError:
            err +=1
            print(json_path)

print('ABS:', abb)
print('err:', err)

  9%|▊         | 9725/113725 [09:52<4:07:19,  7.01it/s] 

In [189]:
print(len( x_data ))
print(len( y_data ))
print(len( drop_t ))
print(len(x_data[-1]))
print(len(y_data))
print(y_data[1])

2194
2194
2187
224
2194
0


In [190]:
import random
remo = sorted(random.sample(drop_t,20000))

for i in range(len(remo)):
    del y_data[remo[-i-1]]
    del x_data[remo[-i-1]]
    
    

In [47]:
print(type(x_data))
import pickle
with open('t_Mu03_x_data.pkl', 'wb') as f:
    pickle.dump(x_data,f)

<class 'list'>


In [192]:
x_data = np.array(x_data)
y_data = np.array(y_data)

In [193]:
x_data.shape

(694, 224, 224, 3)

In [194]:
y_data.shape

(694,)

In [16]:
print(y_data)

[0 0 0 ... 0 0 0]


In [195]:
# a_path = "C:/Users/seonh/Downloads/062.반려동물 질병 진단을 위한 영상 데이터 (근골격계)/01.데이터/Training/augment"

# np.save(a_path +'/'+di+'_x_data_tr_1000.npy', x_data)
# np.save(a_path +'/'+di+'_y_data_tr_1000.npy', y_data)


dir_path = "C:/Users/seonh/Downloads/062.반려동물 질병 진단을 위한 영상 데이터 (근골격계)/01.데이터/Validation/numpy_save"

np.save(dir_path +'/'+di+'_x_data_val_1000.npy', x_data)
np.save(dir_path +'/'+di+'_y_data_val_1000.npy', y_data)
